In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

import glob
import imageio as iio
import numpy as np
import os
import pandas as pd
import random

In [ ]:
SUBJ_LIST = ["subj01", "subj02", "subj03", "subj04", "subj05", "subj06", "subj07", "subj08"]

In [108]:
ROOT_FILE_PATH = "/Users/adminpc-macmini/Library/CloudStorage/GoogleDrive-jeremias.rueck@gmail.com/Meine Ablage/Google Drive - Code/NTHU/final_project/data"

def ReadNpyFilesOfDir(dir_path):
  dic = {}
  index = 1
  for path in glob.glob(dir_path + "/*.npy"):
    print('Load npy file: ' + str(path))
    dic[index] = {
      0: path.split("/")[-1],
      1: np.load(path, allow_pickle = True)
    }
    index += 1
  return dic

def ReadPngFile(dir_path):
  dic = {}
  index = 0
  for path in glob.glob(dir_path + "/*.png"):
    print('Load picture: ' + str(path))
    dic[index] = {
      0: path.split("/")[-1],
      1: iio.imread(path),
      2 : {}
    }
    for j in range(425):
      dic[index][2][j] = {}
      for k in range(425):
         dic[index][1][j][k] = 0.299 * dic[index][1][j][k][0] + 0.587 * dic[index][1][j][k][1] + 0.114 * dic[index][1][j][k][2]
    index += 1
  return dic

In [109]:
#readme
"""
Subject 1-8:

image[0-4999] --> [0] --> name
                  [1] --> RGB([0-455][0-455]pixel)(value:0-255)
                  [2] --> gray scale
                  [3] --> (133 labels) (0 or 1)
                  
fmri[0-4999]  --> [0] --> left
                  [1] --> right

mask[0-31]    --> [0] --> name(type of mask)
                  [1] --> np_array
"""

'\nSubject 1-8:\n\nimage[0-4999] --> [0] --> name\n                  [1] --> RGB([0-455][0-455]pixel)(value:0-255)\n                  [2] --> gray scale\n                  [3] --> (133 labels) (0 or 1)\n                  \nfmri[0-4999]  --> [0] --> left\n                  [1] --> right\n\nmask[0-31]    --> [0] --> name(type of mask)\n                  [1] --> np_array\n'

In [110]:
class ReadObject:
  
  labels = None
  labels_names = None

  def __init__(self, subject):
    self.subject = subject

  def get_info(self):
  
    image_data = pd.read_csv(os.path.join(ROOT_FILE_PATH, "image_infos", f"{self.subject}_infos_train.csv"))
    image_data = image_data.drop(image_data.columns[[0, 1]], axis=1)
    
    self.labels_names = image_data.columns
    self.labels = np.array(image_data.values, dtype=int)
    print('Image count: ' + str(image_data.shape[0]))
    
    label_count = len(image_data.columns)
    print('Label count: ' + str(label_count))
           
    return self.labels_names, self.labels

  fmri = None
  masks = None
  images = None
  ## Mode = "train" | "test"
  def load(self, mode = "train"):
    
    self.masks = ReadNpyFilesOfDir(os.path.join(ROOT_FILE_PATH, self.subject , "roi_masks"))

    if mode == "train":
      self.images = ReadPngFile(os.path.join(ROOT_FILE_PATH, self.subject, "training_split", "training_images"))
      self.fmri = ReadNpyFilesOfDir(os.path.join(ROOT_FILE_PATH, self.subject, "training_split", "training_fmri"))

    elif mode == "test":
      self.images = None
      self.fmri = ReadNpyFilesOfDir(os.path.join(ROOT_FILE_PATH, self.subject, "test_split", "test_fmri"))
          
    return self.images, self.fmri, self.masks

  ## batch_number = probability of
  def split_data(self, batch_number = 1):
    training_set = []
    validation_set = []
    for i in range(5000):
        eat = random.random()
        if eat < batch_number:
            training_set.append(self.image[i])
        else:
            validation_set.append(self.image[i])
    training_set = np.array(training_set)
    validation_set =np.array(validation_set)
    return training_set, validation_set

In [111]:
subject = SUBJ_LIST[0]

combined_data = ReadObject(subject)

In [112]:
label_name, labels = combined_data.get_info()

Image count: 5000
Label count: 133


In [113]:
label_name

Index(['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train',
       'truck', 'boat', 'traffic light',
       ...
       'pavement-merged', 'mountain-merged', 'grass-merged', 'dirt-merged',
       'paper-merged', 'food-other-merged', 'building-other-merged',
       'rock-merged', 'wall-other-merged', 'rug-merged'],
      dtype='object', length=133)

In [114]:
labels.shape

(5000, 133)

In [115]:
images, fmri, masks = combined_data.load()

Load npy file: /Users/adminpc-macmini/Library/CloudStorage/GoogleDrive-jeremias.rueck@gmail.com/Meine Ablage/Google Drive - Code/NTHU/final_project/data/subj01/roi_masks/rh.all-vertices_fsaverage_space.npy
Load npy file: /Users/adminpc-macmini/Library/CloudStorage/GoogleDrive-jeremias.rueck@gmail.com/Meine Ablage/Google Drive - Code/NTHU/final_project/data/subj01/roi_masks/lh.floc-places_space.npy
Load npy file: /Users/adminpc-macmini/Library/CloudStorage/GoogleDrive-jeremias.rueck@gmail.com/Meine Ablage/Google Drive - Code/NTHU/final_project/data/subj01/roi_masks/rh.floc-faces_fsaverage_space.npy
Load npy file: /Users/adminpc-macmini/Library/CloudStorage/GoogleDrive-jeremias.rueck@gmail.com/Meine Ablage/Google Drive - Code/NTHU/final_project/data/subj01/roi_masks/lh.floc-bodies_fsaverage_space.npy
Load npy file: /Users/adminpc-macmini/Library/CloudStorage/GoogleDrive-jeremias.rueck@gmail.com/Meine Ablage/Google Drive - Code/NTHU/final_project/data/subj01/roi_masks/lh.floc-faces_fsaver

KeyboardInterrupt: 

In [ ]:
images

In [ ]:
fmri

In [ ]:
masks

In [ ]:
combined_data.load(subject, temp)

combined_data.read_fmri()

data_train, data_valid = combined_data.splitted_data(0.9)
# np.save('/content/drive/My Drive/Colab Notebooks/data_valid.npy', data_valid)
# np.save('/content/drive/My Drive/Colab Notebooks/data_train.npy', data_train)